### Installing DuckDB and magic_duckdb extension

Download data from here:
https://www.kaggle.com/datasets/catalystcooperative/pudl-project?select=hourly_emissions_epacems.parquet

In [30]:
!python3 -m venv .duckvenv
!source .duckvenv/bin/activate

In [35]:
!pip install duckdb magic_duckdb polars plotly_express nbformat --quiet --user


/usr/lib/python3.11/pty.py:89: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()


/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
Traceback (most recent call last):
  File "/bin/pip", line 11, in <module>
    load_entry_point('pip==20.0.2', 'console_scripts', 'pip')()
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/main.py", line 73, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/commands/__init__.py", line 96, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>",

In [45]:
import duckdb
import pandas as pd
import polars as pl
import plotly_express as px
import pyarrow

%reload_ext magic_duckdb

Read more about Polars here:
https://pola.rs/#

## Analyzing Data with DuckDB

### SQL queries using DuckDB:
To run SQL queries in DuckDB we can directly use ".sql", no need to create a connection to ":memory:" 

Every DataFrame inside this notebook will be instantly available for DuckDB to make SQL queries against.

### Starting with something simple 

In [48]:
df = pd.DataFrame({
    'column1': [1, 2, 3, 4, 5], 
    'column2': ['a', 'b', 'c', 'd', 'e'], 
})

In [49]:
df.head()

,column1,column2
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


In [50]:
duckdb.sql('''
SELECT * 
FROM df
''')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

- DuckDB was able to query our Pandas DataFrame without any extra steps

- Let's go directly from "FROM..." leaving  "SELECT *" as a legacy

In [51]:
duckdb.sql('''
FROM df 
''')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

In [52]:
duckdb.sql('FROM df')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

In [16]:
# Conectando ao DuckDB e registrando o dataframe



# Exemplo de dataframe
df = pd.DataFrame({"col1": [1, 2], "col2": [3, 4]})
con = duckdb.connect()
con.register("df", df)
con.execute("SHOW TABLES").fetchall()
result = con.execute("SELECT * FROM df").fetchall()
print(result)
%reload_ext magic_duckdb


[(1, 3), (2, 4)]


We installed duckdb magic earlier and can now avoid repeating `duckdb.sql` and use:
- `%dql` for single line queries 
&
- `%%dql` for multiple lined queries instead

In [53]:
%%dql -t show 
SHOW TABLES

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘



In [56]:
%%dql 
SELECT * FROM df

ValueError: Error executing SELECT * FROM df in DuckDB

Note that by using the magic_duckdb extension, our queries return a Pandas DataFrame, meaning we are still making use of the "lazyness" of DuckDB during the query, but then immediately bringing the query result entirely to memory.

To avoid this, we can set the type of return by using "-t" followed by the type, choosing from "df", "arrow", "pl", "describe", "show" and "relation".

In [ ]:
%%dql -t show 
FROM df
SELECT *

We can also change the default behavior by passing the "cell magic + type" without a query.

Let's set the extension default to DuckDB's default using the type "show".

In [11]:
%dql -t show

In [ ]:
%%dql
FROM df
SELECT *

### Doing 'Big Data' analysis. Bigger than RAM!
- DuckDB can also connect directly to files like CSV/Parquet

- We have a __huge__ file with emmissions: let's start working with it
- For starters, let's take advantage of DuckDB's "lazyness" and count the rows while avoiding loading all data into machine's memory

In [ ]:
!pwd

In [ ]:
duckdb.sql('''
SELECT COUNT(*) 
FROM 'hourly_emissions_epacems.parquet'
'''
)

We could instantly count all those 897 million rows. It's like querying a database.

Let's now do 3 cool things: 
- get a number of descriptive statistics using avg & max filtering data from just one state
- do the same thing using Pandas
-compare execution time

In [ ]:
%%time
%%dql
SELECT avg(co2_mass_tons) as average_co2_emmission, max(co2_mass_tons) as max_co2_emmission
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'

In [ ]:
%%time
df = pd.read_parquet('hourly_emissions_epacems.parquet', engine='pyarrow', columns=['state', 'co2_mass_tons'])
print(df[df['state']=='CO']['co2_mass_tons'].mean(), df[df['state']=='CO']['co2_mass_tons'].max())

we can also display descriptive stats as a polars DataFrame

In [ ]:
%%time
%%dql -t pl
SUMMARIZE 
SELECT * 
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'

Now let's group some data so we can create a visualization.

To store the query output into a variable, we can use "-o" followed by the variable name. Let's save it as a Polars DataFrame to the variable "df_gb".

In [ ]:
%%time
%%dql -t pl -o df_gb
SELECT 
    year, 
    hour(operating_datetime_utc) AS hour, 
    SUM(co2_mass_tons) AS co2, 
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'
GROUP BY ALL